In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
data = pd.read_csv('data.csv')
df_w_genre=pd.read_csv('data_w_genres.csv')


In [ ]:
data.isnull().sum()

In [ ]:
df_w_genre.isnull().sum()

In [ ]:
merged_data = pd.merge(data, df_w_genre, on=['mode', 'acousticness', 'danceability', 'duration_ms', 'energy',
                                            'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
                                            'valence', 'popularity', 'key'])

In [ ]:
numeric_genre_data = df_w_genre.select_dtypes(include = 'number')

In [ ]:
from sklearn.preprocessing import StandardScaler

genrew_scaler = StandardScaler()
scaled_genre_data = genrew_scaler.fit_transform(numeric_genre_data)

In [ ]:
from sklearn.cluster import KMeans
genre_artist_cluster = KMeans(n_clusters = 20)
genre_artist_cluster.fit(scaled_genre_data)

In [ ]:
genre_artist_cluster_labels = genre_artist_cluster.labels_
df_w_genre['clusters'] = genre_artist_cluster_labels

In [ ]:
# Reducing the dimension of scaled data using TSNE for graph plotting
from sklearn.manifold import TSNE

genre_reducer = TSNE(n_components = 2)
reduced_genre_data = genre_reducer.fit_transform(scaled_genre_data)

#creating new dataset for the reduced dimension values
new_genre_dataset = pd.DataFrame(columns = ['x-axis', 'y-axis'], data = reduced_genre_data)
new_genre_dataset['genres'] = df_w_genre['genres']
new_genre_dataset['clusters'] = df_w_genre['clusters']


In [ ]:
import plotly.express as px
genre_cluster_graph = px.scatter(new_genre_dataset, x = 'x-axis', y = 'y-axis', color = 'clusters', hover_data = ['x-axis', 'y-axis', 'genres'])
genre_cluster_graph.show()

In [ ]:
numeric_song_data = data.select_dtypes(include = 'number')

In [ ]:
song_scaler = StandardScaler()
scaled_song_data = song_scaler.fit_transform(numeric_song_data)

In [ ]:
song_cluster = KMeans(n_clusters = 20)
song_cluster.fit(scaled_song_data)

In [ ]:
song_cluster_labels = song_cluster.labels_
data['clusters'] = song_cluster_labels


In [ ]:
from sklearn.decomposition import PCA

song_reducer = PCA(n_components = 2)
reduced_song_data = song_reducer.fit_transform(scaled_song_data)

In [ ]:
new_song_dataset = pd.DataFrame(columns = ['x-axis', 'y-axis'], data = reduced_song_data)
new_song_dataset['name'] = data['name']
new_song_dataset['clusters'] = data['clusters']

In [ ]:
song_graph = px.scatter(new_song_dataset, x = 'x-axis', y = 'y-axis', color = 'clusters', hover_data = ['x-axis', 'y-axis', 'clusters'])
song_graph.show()

In [ ]:
numeric_artists_data = df_w_genre.select_dtypes(include = 'number')

In [ ]:
from sklearn.preprocessing import StandardScaler

artist_scaler = StandardScaler()
scaled_artist_data = artist_scaler.fit_transform(numeric_artists_data)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
artist_cluster = KMeans(n_clusters = 10)
artist_cluster.fit(scaled_artist_data)

In [ ]:
artist_cluster_labels = artist_cluster.labels_
df_w_genre['clusters'] = artist_cluster_labels

In [ ]:
# Reducing the dimension of scaled data using TSNE for graph plotting
from sklearn.manifold import TSNE

genre_reducer = TSNE(n_components = 2)
reduced_genre_data = genre_reducer.fit_transform(scaled_artist_data)

#creating new dataset for the reduced dimension values
new_artist_dataset = pd.DataFrame(columns = ['x-axis', 'y-axis'], data = reduced_genre_data)
new_artist_dataset['artists'] = df_w_genre['artists']
new_artist_dataset['clusters'] = df_w_genre['clusters']

In [ ]:
# plotting the cluster graph
import plotly.express as px
artist_cluster_graph = px.scatter(new_artist_dataset, x = 'x-axis', y = 'y-axis', color = 'clusters', hover_data = ['x-axis', 'y-axis', 'artists'])
artist_cluster_graph.show()

In [ ]:
def recommend1(best_song):
    song_cluster = data.loc[data['name'] == artist_name, 'clusters'].values[0]
    artist_cluster = data.loc[data['name'] == artist_name, 'artists'].values[0]
    genre_cluster = data.loc[data['name'] == artist_name,'genres'].values[0]
    similar_cluster_songs = data.loc[data['clusters'] == song_cluster]
    similar_artist_songs = similar_cluster_songs.loc[similar_cluster_songs['artists'] == song_artist]
    similar_genre_songs = similar_artist_songs.loc[similar_artist_songs['genres']== song_artist]
    recommending_songs = similar_artist_songs.loc[similar_artist_songs['name'] != artist_name]
    top_recommending_songs = recommending_songs.nlargest(5, 'popularity')
    for index,row in top_recommending_songs.iterrows():
        name_of_song = row['name']
        ls.append(name_of_song)
    return ls